# **Marketing Analytics**
# **Homework Assignment 1**
# **Bradley Ge**

## Import libraries

In [1]:
import pandas as pd 
import numpy as np

## Import Data

In [2]:
customerdf = pd.read_csv("/Users/bradley/Desktop/Marketing Analytics/HW1/data/cust_data.csv")
transactiondf = pd.read_csv("/Users/bradley/Desktop/Marketing Analytics/HW1/data/transaction_data.csv")

## Check Data

In [3]:
customerdf.head()

,Customer ID,State,Treatment Group
0,0,VA,No Mail
1,1,VA,Mail
2,2,VA,Mail
3,3,OH,Mail
4,4,OH,Mail


In [4]:
transactiondf.head()

,Customer ID,Order Date,Profit
0,1,2019-02-24,27.94
1,2,2019-02-11,196.33
2,2,2019-02-13,79.19
3,2,2019-02-21,255.27
4,2,2019-04-14,126.26


In [5]:
customerdf.shape

(12000, 3)

In [6]:
transactiondf.shape

(29659, 3)

In [7]:
customerdf.dtypes

Customer ID         int64
State              object
Treatment Group    object
dtype: object

In [8]:
transactiondf.dtypes

Customer ID      int64
Order Date      object
Profit         float64
dtype: object

In [9]:
transactiondf['Order Date'] = pd.to_datetime(transactiondf['Order Date'])
transactiondf.dtypes

Customer ID             int64
Order Date     datetime64[ns]
Profit                float64
dtype: object

## Merge Data

In [10]:
df = pd.merge(transactiondf,customerdf,on= "Customer ID",how="left")
df.head()

,Customer ID,Order Date,Profit,State,Treatment Group
0,1,2019-02-24,27.94,VA,Mail
1,2,2019-02-11,196.33,VA,Mail
2,2,2019-02-13,79.19,VA,Mail
3,2,2019-02-21,255.27,VA,Mail
4,2,2019-04-14,126.26,VA,Mail


In [11]:
df.shape

(29659, 5)

In [12]:
df.dtypes

Customer ID                 int64
Order Date         datetime64[ns]
Profit                    float64
State                      object
Treatment Group            object
dtype: object

## Question 1: Preprocess the data

In [13]:
treatment_date = pd.to_datetime('2019-03-02')
pre_period = (df['Order Date'] >= treatment_date - pd.Timedelta(days=60)) & (df['Order Date'] < treatment_date)
post_period = (df['Order Date'] >= treatment_date) & (df['Order Date'] <= treatment_date + pd.Timedelta(days=60))

In [14]:
pre_treatment_profit_sum = df.loc[pre_period].groupby('Customer ID')['Profit'].sum().reset_index().rename(columns={'Profit': 'Profit 60 days before the treatment date'}).fillna(0)
post_treatment_profit_sum = df.loc[post_period].groupby('Customer ID')['Profit'].sum().reset_index().rename(columns={'Profit': 'Profit 60 days after the treatment date'}).fillna(0)

In [15]:
df_1 = pd.merge(customerdf,pre_treatment_profit_sum,on='Customer ID',how="left").fillna(0)
df_Q1 = pd.merge(df_1,post_treatment_profit_sum,on='Customer ID',how="left").fillna(0)
df_Q1.head(10)

,Customer ID,State,Treatment Group,Profit 60 days before the treatment date,Profit 60 days after the treatment date
0,0,VA,No Mail,0.00,0.00
1,1,VA,Mail,27.94,0.00
2,2,VA,Mail,530.79,126.26
3,3,OH,Mail,0.00,0.00
4,4,OH,Mail,54.72,99.91
5,5,GA,No Mail,84.57,264.63
6,6,OH,Mail,68.01,317.61
7,7,OH,Mail,0.00,0.00
8,8,OH,Mail,0.00,83.15
9,9,VA,Mail,0.00,65.59


In [16]:
df_Q1.describe()

,Customer ID,Profit 60 days before the treatment date,Profit 60 days after the treatment date
count,12000.00000,12000.000000,12000.000000
mean,5999.50000,77.331597,119.349886
std,3464.24595,229.006802,337.118820
min,0.00000,0.000000,0.000000
25%,2999.75000,0.000000,0.000000
50%,5999.50000,5.560000,14.545000
75%,8999.25000,65.220000,103.285000
max,11999.00000,7372.070000,9649.510000


In [17]:
df_Q1["Profit 60 days before the treatment date"].agg(['mean','std'])

mean     77.331597
std     229.006802
Name: Profit 60 days before the treatment date, dtype: float64

In [18]:
df_Q1["Profit 60 days after the treatment date"].agg(['mean','std'])

mean    119.349886
std     337.118820
Name: Profit 60 days after the treatment date, dtype: float64

## Question 2: Before and After

In [19]:
# a)Treatment Effect = Average Profit Before - Average Profit After
# = 119.35 - 77.33
round(119.35 - 77.33,2)

42.02

In [20]:
# b) Yes. If the results are statistically significant, and we consider that it is relatively valid externally, 
# we can launch this campaign.

In [69]:
#C) 42.02 * 300,000 
(42.02-20)*300000

6606000.000000001

## Question 3: Randomization Check

In [22]:
# a) Chi-Square Test.(to test if there's difference in state between treatment and control group)

In [23]:
from scipy.stats import chi2_contingency

In [24]:
contingency_table = pd.crosstab(df_Q1['State'], df_Q1['Treatment Group'])
contingency_table

Treatment Group,Mail,No Mail
State,,
GA,2171,2154
OH,2035,2067
VA,1794,1779


In [25]:
# H0: The distribution of the categorical variable is the same in both groups.
# H1: The distribution of the categorical variable is different in both groups.
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"Chi-square test p-value for 'State': {round(p,2)}")

Chi-square test p-value for 'State': 0.83


In [26]:
# This conclude that there's no statistical significance between state and treatment group, 
# which means the distribution of participants across different states is similar between the two groups, 
# indicating that the randomization process was successful in this aspect.

In [27]:
# b) t-test

In [28]:
from scipy.stats import ttest_ind

In [29]:
treatment_profit_before = df_Q1[df_Q1['Treatment Group'] == 'Mail']['Profit 60 days before the treatment date']
control_profit_before = df_Q1[df_Q1['Treatment Group'] == 'No Mail']['Profit 60 days before the treatment date']

In [30]:
stat, p_value = ttest_ind(treatment_profit_before, control_profit_before, equal_var=False)
print(f'T-statistic: {round(stat,2)}, P-value: {round(p_value,2)}')

T-statistic: -1.09, P-value: 0.27


In [31]:
# p-value > 0.05, which means any differences observed after the treatment are more likely due to the 
# treatment itself rather than pre-existing differences

In [32]:
# c) These two test proves that there's no statistical significance between either state or profit 60 days before
# the treatment with tereatment group, so they are all random. With this in mind, we can conclude that the result
# of the treatment is not affected by any other pre-existing difference but the effect of this treatment.

## Question 4: Average Treatment Effect

In [33]:
#a)
treatment_profit_after = df_Q1[df_Q1['Treatment Group'] == 'Mail']['Profit 60 days after the treatment date']
control_profit_after = df_Q1[df_Q1['Treatment Group'] == 'No Mail']['Profit 60 days after the treatment date']
# Calculating the mean profit for each group
mean_treatment_profit = treatment_profit_after.mean()
mean_control_profit = control_profit_after.mean()
mean_treatment_before = treatment_profit_before.mean()
mean_control_before = control_profit_before.mean()
# Calculating the Average Treatment Effect
ATE = mean_treatment_profit - mean_treatment_before
round(ATE,2)

53.17

In [34]:
# b)
t_stat, p_value = ttest_ind(treatment_profit_after, treatment_profit_before, equal_var=False)
n_treatment = len(treatment_profit_after)
n_control = len(treatment_profit_before)
sd_treatment = np.std(treatment_profit_after, ddof=1)
sd_control = np.std(treatment_profit_before, ddof=1)

SE = np.sqrt(sd_treatment**2/n_treatment + sd_control**2/n_control)
print(f"P-value: {round(p_value,5)}")
print(f"Standard Error: {round(SE,2)}")

P-value: 0.0
Standard Error: 5.62


In [35]:
# c)
# The hypothesis shows that sending a catalog has a statistically significant effect on profit, so I would.

In [61]:
# d)
round((53.17-20)*300000,2)

9951000.0

In [37]:
# e)
# Idearly, ITT and ATT should be the same, but the result shows that ATT is higher than ITT due to the noise in reality.
# Overall, they all reflect that launching this campaign will generate profit.

## Question 5: Difference in Difference

In [38]:
# a)
DiD = (mean_treatment_profit - mean_treatment_before) - (mean_control_profit - mean_control_before)
round(DiD,2)

22.31

In [39]:
# b)
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [40]:
df_long_pre = df_Q1[['Customer ID', 'State', 'Treatment Group', 'Profit 60 days before the treatment date']].copy()
df_long_pre['Time'] = 'Pre'
df_long_pre.rename(columns={'Profit 60 days before the treatment date': 'Profit'}, inplace=True)

df_long_post = df_Q1[['Customer ID', 'State', 'Treatment Group', 'Profit 60 days after the treatment date']].copy()
df_long_post['Time'] = 'Post'
df_long_post.rename(columns={'Profit 60 days after the treatment date': 'Profit'}, inplace=True)

df_long = pd.concat([df_long_pre, df_long_post], axis=0)
df_long['Post'] = (df_long['Time'] == 'Post').astype(int)
df_long['Treatment'] = (df_long['Treatment Group'] == 'Mail').astype(int)
df_long['Interaction'] = df_long['Post'] * df_long['Treatment']

In [41]:
X = sm.add_constant(df_long[['Post', 'Treatment', 'Interaction']])
# Define the dependent variable
Y = df_long['Profit']

# Fit the model
model = sm.OLS(Y, X).fit()

# Print the summary results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 Profit   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     46.58
Date:                Mon, 12 Feb 2024   Prob (F-statistic):           5.25e-30
Time:                        13:29:49   Log-Likelihood:            -1.6997e+05
No. Observations:               24000   AIC:                         3.400e+05
Df Residuals:                   23996   BIC:                         3.400e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          79.6146      3.720     21.404      

In [42]:
# p value = 0.003
# s.e = 7.439

In [43]:
# c)
# the result is statistically significant, so we can conclude that implementing the treatment more broadly 
# could yield positive outcomes.

In [62]:
# d)
round((22.31-20)*300000,2)

693000.0

In [99]:
round((36.89-20)*300000,2)

5067000.0

In [45]:
# e)
# The DiD estimate considers the parallel trend assumption, which means the estimate exclude the amount of increase
# in the control group as time goes by, therefore, the value is lower than ATT.

## Question 6: Basic Targeting

In [65]:
import warnings
warnings.filterwarnings("ignore")

In [81]:
# a)
df_GA = df_Q1[df_Q1['State']== "GA"]
df_GA = df_GA[df_GA["Treatment Group"] == 'Mail']
df_GA["diff"] = df_GA['Profit 60 days after the treatment date'] - df_GA['Profit 60 days before the treatment date']
ATE_GA = df_GA['diff'].mean()
ATE_GA

74.52027176416398

In [83]:
df_OH = df_Q1[df_Q1['State']== "OH"]
df_OH = df_OH[df_OH["Treatment Group"] == 'Mail']
df_OH["diff"] = df_OH['Profit 60 days after the treatment date'] - df_OH['Profit 60 days before the treatment date']
ATE_OH = df_OH['diff'].mean()
ATE_OH

51.238447174447174

In [86]:
df_VA = df_Q1[df_Q1['State']== "VA"]
df_VA = df_VA[df_VA["Treatment Group"] == 'Mail']
df_VA["diff"] = df_VA['Profit 60 days after the treatment date'] - df_VA['Profit 60 days before the treatment date']
ATE_VA = df_VA['diff'].mean()
ATE_VA

29.533846153846156

In [87]:
print(f'ATE for GA is {round(ATE_GA,2)}')
print(f'ATE for OH is {round(ATE_OH,2)}')
print(f'ATE for VA is {round(ATE_VA,2)}')

ATE for GA is 74.52
ATE for OH is 51.24
ATE for VA is 29.53


In [88]:
# b) Hypothesis test 
treatment_before_GA = df_Q1[df_Q1['State'] == 'GA']['Profit 60 days before the treatment date']
treatment_after_GA = df_Q1[df_Q1['State'] == 'GA']['Profit 60 days after the treatment date']
treatment_before_OH = df_Q1[df_Q1['State'] == 'OH']['Profit 60 days before the treatment date']
treatment_after_OH = df_Q1[df_Q1['State'] == 'OH']['Profit 60 days after the treatment date']
treatment_before_VA = df_Q1[df_Q1['State'] == 'VA']['Profit 60 days before the treatment date']
treatment_after_VA = df_Q1[df_Q1['State'] == 'VA']['Profit 60 days after the treatment date']

In [89]:
t_stat, p_value = ttest_ind(treatment_before_GA, treatment_after_GA, equal_var=False)
print(f'T-statistic: {round(stat,2)}, P-value: {round(p_value,5)}')

T-statistic: -1.09, P-value: 0.0


In [91]:
t_stat, p_value = ttest_ind(treatment_before_OH, treatment_after_OH, equal_var=False)
print(f'T-statistic: {round(stat,2)}, P-value: {round(p_value,5)}')

T-statistic: -1.09, P-value: 0.0


In [92]:
t_stat, p_value = ttest_ind(treatment_before_VA, treatment_after_VA, equal_var=False)
print(f'T-statistic: {round(stat,2)}, P-value: {round(p_value,5)}')

T-statistic: -1.09, P-value: 0.0


In [54]:
# Results are statistically significant, and the profit is positive, so I will launch the campaign.

In [93]:
# c)
prob_GA = df_Q1[df_Q1['State'] == 'GA']['Customer ID'].count()/12000

In [56]:
prob_OH = df_Q1[df_Q1['State'] == 'OH']['Customer ID'].count()/12000

In [57]:
prob_VA = df_Q1[df_Q1['State'] == 'VA']['Customer ID'].count()/12000

In [95]:
profit_GA = (ATE_GA-20) * 300000 * prob_GA
profit_OH = (ATE_OH-20) * 300000 * prob_OH
profit_VA = (ATE_VA-20) * 300000 * prob_VA

In [96]:
print(f'Estimated Profit if launch in GA is {round(profit_GA)}')
print(f'Estimated Profit if launch in OH is {round(profit_OH)}')
print(f'Estimated Profit if launch in VA is {round(profit_VA)}')

Estimated Profit if launch in GA is 5895004
Estimated Profit if launch in OH is 3203503
Estimated Profit if launch in VA is 851611


## Question 7: Recommendation

In [ ]:
# Given the results, we are confident that launching the campaign in Georgia, Ohio, and Virginia states will 
# bring us profit, and Georgia will bring the most among all three states. The average treatment effect shows 
# that the profit will be $ 9,951,000, where GA brings the most profit ($ 5,895,004), followed by OH ($ 3,203,503), 
# and VA($ 851,611). However, according to the difference in difference estimate, even though the expected profit 
# assuming parallel trend is $ 693,000, we are 95% confident that the profit will fall between $ -3,681,600 
# and $ 5,067,000, which include negative profit. Still, the result is based on the parallel trend assumption, 
# and the expected profit is still above 0.